## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
# checking if it is empty
# dbutils.fs.rm(BRONZE_CHECKPOINT, True)
# dbutils.fs.rm(BRONZE_DELTA, True)
# dbutils.fs.rm(SILVER_CHECKPOINT, True)
# dbutils.fs.rm(SILVER_DELTA, True)
# dbutils.fs.rm(GOLD_CHECKPOINT, True)
# dbutils.fs.rm(GOLD_DELTA, True)

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ENTER YOUR CODE HERE

# Q1
spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)

# Q2
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.functions import (
    count, isnan, col, when, current_timestamp,
    input_file_name, desc, explode, expr,
    to_timestamp, regexp_replace
)
import matplotlib.pyplot as plt
import mlflow
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay
)



## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE

# Q1
df = get_source_listing_df()

# Q2
number_of_rows = df.count()
print("The number of source files in S3 bucket are "+str(number_of_rows))

# Q3
file_name = df.loc[0,'File Name'] # getting the file name 
show_s3_file_contents(file_name) # printing the file contents

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE

# Q1
schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True),
    StructField("processing_time", TimestampType(), True)
])



In [0]:
# Q
df_stream = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaLocation", "/mnt/delta/schema")
    .schema(schema)
    .load(TWEET_SOURCE_PATH)
    # .load('/dbfs/mnt/test-tweet-sample')
)

In [0]:
df_stream = df_stream.withColumn("processing_time", current_timestamp())
df_stream = df_stream.withColumn("source_file", input_file_name())

In [0]:
bronze_stream = (
    df_stream.writeStream
    .format("delta")
    .option("mergeSchema", "true")
    .outputMode("append")
    .option("checkpointLocation", BRONZE_CHECKPOINT)
    .queryName("bronze_stream")
    .trigger(availableNow=True)
    .start(BRONZE_DELTA)
)

In [0]:
bronze_stream.awaitTermination()

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ENTER YOUR CODE HERE

# Q1
df_bronze = spark.read.format("delta").load(BRONZE_DELTA)
number_of_tweets = df_bronze.count()
print("The number of tweets in the bronze table are "+str(number_of_tweets))

In [0]:
# Q2
print("The number of NULL values for every column:")

df_bronze.select([count(when(col(c).isNull(), c)).alias(c) for c in df_bronze.columns]).show()

print("The number of NAN values for every column:")
df_bronze.select([count(when(isnan(col(c)), c)).alias(c) for c in df_bronze.columns if c !='processing_time']).show()

In [0]:
#  Q3
twitter_count = df_bronze.groupBy("user").count().orderBy(desc("count"))
twitter_count.show(50)

In [0]:
# Q6
import seaborn as sns

twitter_count = twitter_count.limit(20)  # Extract top 20 rows

# Convert the Spark DataFrame to a Pandas DataFrame for plotting
twitter_count_pd = twitter_count.toPandas()

# Set the style and color palette
sns.set_style("whitegrid")
sns.set_palette("pastel")

# Plotting the bar chart
plt.figure(figsize=(10, 6))
barplot = sns.barplot(x='user', y='count', data=twitter_count_pd)
plt.xlabel('User')
plt.ylabel('Tweet Count')
plt.title('Top 20 Twitter Users by Tweet Count')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability

# Adding annotations with the count values on top of each bar
for index, row in twitter_count_pd.iterrows():
    barplot.text(index, row['count'] + 0.5, row['count'], color='black', ha='center')

plt.tight_layout()

In [0]:
# Q4

df_mentions = df_bronze.withColumn("mentions", when(col("text").contains("@"), 1).otherwise(0))

mentions_count = df_mentions.filter(col("mentions") == 1).count()
no_mentions_count = df_bronze.count() - mentions_count

# Print the results
print("The number of tweets with mentions "+str(mentions_count))
print("The number of tweets with no mentions "+str(no_mentions_count))

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY") 

bronze_read_stream = (
    spark.readStream
    .format('delta')
    .load(BRONZE_DELTA)
)

In [0]:

# Transformations

# converting the timestamp string to timestamp datatype
df_silver = bronze_read_stream.withColumn("timestamp", to_timestamp("date", "E MMM dd HH:mm:ss z yyyy"))

# creating the mentions column
df_silver = df_silver.withColumn("mentions", expr("regexp_extract_all(text, '(?<=^|(?<=[^a-zA-Z0-9-_\\\\.]))@([A-Za-z]+[A-Za-z0-9_]+)', 0)"))
df_silver = df_silver.withColumn("mention", explode(col("mentions"))).drop("mentions")

# creating the cleaned text
df_silver = df_silver.withColumn("cleaned_text", regexp_replace("text", "(@\\w+)", ""))

# dropping the extra columns
df_silver = df_silver.drop("date", "user", "text", "source_file", "processing_time")

In [0]:

silver_stream = df_silver.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", SILVER_CHECKPOINT) \
    .trigger(availableNow=True) \
    .queryName('silver_stream') \
    .start(SILVER_DELTA)


## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# ENTER YOUR CODE HERE
silver_read_stream = (
    spark.readStream
    .format('delta')
    .load(SILVER_DELTA)
)

In [0]:
# Model
model_uri = f"models:/{MODEL_NAME}/Production"
model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

In [0]:
# SIlver to GOld transformation

# Add a temporary column for predicted sentiment using a model based on cleaned text
df_gold = silver_read_stream.withColumn("temp_predicted_sentiment", model(col("cleaned_text")))

# Compute and assign predicted sentiment scores as integer percentages
df_gold = df_gold.withColumn("predicted_score", (col("temp_predicted_sentiment.score") * 100).cast("int"))

# Assign predicted sentiment labels based on the temporary column
df_gold = df_gold.withColumn("predicted_sentiment", col("temp_predicted_sentiment.label"))

# Create a binary sentiment identifier based on the original sentiment column
df_gold = df_gold.withColumn("sentiment_id", when(col("sentiment") == "positive", 1).otherwise(0))

# Create a binary predicted sentiment identifier based on the predicted sentiment column
df_gold = df_gold.withColumn("predicted_sentiment_id", when(col("predicted_sentiment") == "POS", 1).otherwise(0))

gold_df = df_gold.drop("temp_predicted_sentiment")

In [0]:
gold_stream = (
    df_gold.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .trigger(availableNow=True)
    .queryName("siver_stream")
    .start(GOLD_DELTA)
)

In [0]:
gold_stream.awaitTermination()

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ENTER YOUR CODE HERE

gold_df = spark.read.format('delta').load(GOLD_DELTA)


mlflow.start_run()
model_name = MODEL_NAME
mlflow_version = mlflow.__version__


TP = gold_df.where((col("predicted_sentiment_id") == 1) & (col("sentiment_id") == 1)).count()
FP = gold_df.where((col("predicted_sentiment_id") == 1) & (col("sentiment_id") == 0)).count()
FN = gold_df.where((col("predicted_sentiment_id") == 0) & (col("sentiment_id") == 1)).count()
TN = gold_df.where((col("predicted_sentiment_id") == 0) & (col("sentiment_id") == 0)).count()


# Calculate precision, recall, and F1 score
precision = TP / (TP + FP) if TP + FP != 0 else 0
recall = TP / (TP + FN) if TP + FN != 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

# Store the precision, recall, and F1-score as MLflow metrics 
mlflow.log_metric("precision", precision)
mlflow.log_metric("recall", recall)
mlflow.log_metric("f1_score", f1_score)

mlflow.end_run()

In [0]:
mlflow.end_run()

In [0]:
# Confusion matrix
y_true = gold_df.select("sentiment_id").collect()
y_pred = gold_df.select("predicted_sentiment_id").collect()
conf_mat = confusion_matrix(y_true, y_pred)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=['Negative', 'Positive'])
heatmap = disp.plot(cmap='viridis', ax=ax)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Confusion Matrix', fontsize=14)
plt.show()

# Add color bar if ax.collections is not empty
if len(ax.collections) > 0:
    # Add color bar
    cbar = ax.collections[0].colorbar
    cbar.set_label('Counts', fontsize=10)
    cbar.ax.tick_params(labelsize=10)

# Save the fancy confusion matrix plot as an image
plt.savefig("confusion_matrix_fancy.png", bbox_inches='tight')
plt.close()

In [0]:

mlflow.log_artifact("confusion_matrix.png")

mlflow.log_param("model_name", model_name)
mlflow.log_param("mlflow_version", mlflow_version)

deltaTable = DeltaTable.forPath(spark, GOLD_DELTA)

history_df = deltaTable.history() \
    .select("version") \
    .orderBy("version", ascending=False)

delta_table_version = history_df.collect()[0][0]

mlflow.log_param("delta_table_version", delta_table_version)
mlflow.end_run()

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# Q1

mentions_gold = gold_df.filter(col("mention").isNotNull()).count()
print("The total numbe rof mentions in the gold data is "+str(mentions_gold))

In [0]:
temp_df = gold_df.groupBy("mention").agg(
    count(when(col("sentiment") == "neutral", True)).alias("Neutral"),
    count(when(col("sentiment") == "positive", True)).alias("Positive"),
    count(when(col("sentiment") == "negative", True)).alias("Negative"),
    count("mention").alias("total_mentions")
).orderBy("total_mentions", ascending=False)

In [0]:
positive_mentions_20 = temp_df.orderBy("Positive", ascending=False).limit(20)
negative_mentions_20 = temp_df.orderBy("Negative", ascending=False).limit(20)

In [0]:
import seaborn as sns

def plot_mentions(df, title, sentiment):
    data = df.select("mention", sentiment).collect()
    mentions = [x["mention"] for x in data]
    counts = [x[sentiment] for x in data]
    
    # Set Seaborn style and color palette
    sns.set_style("whitegrid")
    sns.set_palette("pastel")

    # Create the bar plot
    plt.figure(figsize=(10, 5))
    barplot = sns.barplot(x=mentions, y=counts)
    plt.xlabel('Mentions')
    plt.ylabel('Counts')
    plt.title(title)
    plt.xticks(rotation=45)
    
    # Add annotations with count values on top of each bar
    for index, count in enumerate(counts):
        plt.text(index, count + 0.5, str(count), ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

# Plot for top 20 positive mentions
plot_mentions(positive_mentions_20, 'Top 20 Mentions with Positive Sentiment', 'Positive')

# Plot for top 20 negative mentions
plot_mentions(negative_mentions_20, 'Top 20 Mentions with Negative Sentiment', 'Negative')



## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE

# from utlities
show_active_streams()
stop_all_streams()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

elapsed_time = END_TIME - START_TIME
minutes = int(elapsed_time // 60)
seconds = elapsed_time % 60
print("Time Taken: {} minutes and {:.2f} seconds".format(minutes, seconds))


## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE

Spill: It refers to the process of writing intermediate data to disk during the execution of a computation job. This can happen when the amount of data being processed exceeds the available memory resources. 
In the Spark UI for my project, I do not see a Spill as shown in the below screenshot 
<img src="https://drive.google.com/thumbnail?id=1wsEA_xir9Tmvcb_3HKtgsuRgu5jiOw1O&sz=w2000">


I would have expected it to show up in the Spark UI, as mentioned in the following picture I have taken from Medium (https://medium.com/road-to-data-engineering/spark-performance-optimization-series-2-spill-685126e9d21f#:~:text=→%20On%20the%20details%20page%20of%20a%20specific%20stage%20in%20Spark%20UI.&text=→%20Also%20you%20can%20see,query%20plan%20or%20query%20details.)
<img src="https://drive.google.com/thumbnail?id=1SrfA_lhYBKW2XawBvbNIjzVVUdQiDqDv&sz=w2000">

The memory alloted to us in the cluster was enough to handle the 200000 tweets in the S3 bucket.


Skew: It refers to data skew. Data skew occurs when the data distribution is uneven across the processing nodes or partitions, leading to certain nodes or partitions having significantly more data to process than others. 
We have set the spark.sql.shuffle.partitions to spark.sparkContext.defaultParallelism that can help mitigate data skew to some extent by ensuring that the number of partitions created during shuffling aligns with the available resources (cores) in the cluster.

We have these different jobs with differrent processing time as shown below:
<img src="https://drive.google.com/thumbnail?id=1SgHWSkLn8o3m3NEIkPb6j4aYexfk04B7&sz=w2000">

We can see in the following snapshot that the median is 32 min and the max is 42 min, which is less difference. We can say that it does not have skew. 

Taking an example from Medium - 
<img src="https://drive.google.com/thumbnail?id=1Vw0QAePI8_Xf6Q_fQEJiGp9x8W1LDfpA&sz=w2000">
The median is 50 ms and the max is 7 sec therefore it has a lot of skew.

Shuffle: The operations involve the redistribution of data across partitions or nodes in a cluster during certain transformations and actions. This redistribution is necessary for tasks like joins, aggregations, and sorting, where data from multiple partitions or nodes needs to be combined or reorganized.

As we can in the following snapshot we have shuffle:
<img src="https://drive.google.com/thumbnail?id=1SD_gVv1vLPSpqjznjo6XAK6MAE06g2Nn&sz=w2000">

This can be attributed to the groupBy, OrderBy, and Aggregate functions which are wide transformation. 
To enhance data movement efficiency among partitions, we can adjust the number of shuffle partitions to be a multiple of the available core count.

Storage: Storing data as many small files can lead to inefficiencies in storage and processing. This is known as the "small files problem." By optimizing disk storage format, reducing the number of small files, organizing storage locations effectively, and leveraging efficient file formats, Databricks users can improve storage efficiency.

I am not able to see the storage information in the Spark UI as shown below:

<img src="https://drive.google.com/thumbnail?id=1wAUOVRT9wUrl2Z5Pp99-G1WS2ay7Y1_N&sz=w2000">

As given in the snapshot below taken from Apache Spark website:
<img src="https://drive.google.com/thumbnail?id=1EggjU34b3wRXE9xkX0Tl7TcwhKJp3zs9&sz=w2000">

We can find two RDDs listed in the Storage tab and basic information like storage level, number of partitions and memory overhead are provided. 

The S3 bucket stores a large number of small files but they are stored efficiently in the bronze, silver and gold tables.

Serialization: It refers to the process of serializing or packaging code segments (such as functions, closures, or variables) in a format that can be distributed and executed across the nodes in a distributed computing environment.

The serialized tasks can be seen in the below snapshot:
<img src="https://drive.google.com/thumbnail?id=1pLOQXKMNUFT6bYHVZOQHbql2Io7TTIzg&sz=w2000">

The above directed acyclic graph (DAG) of this stage has vertices which represent the RDDs or DataFrames and the edges represent an operation to be applied. 

The event timeline with different tasks and their respective run times is shown below:
<img src="https://drive.google.com/thumbnail?id=18ptUwl5cgEqrTicl2KYfeLy4_cq10rcb&sz=w2000">
